# Deep Reinforcement Learning
In this lab we will implement and train an agent that uses deep learning to play balance the stick in `CartPole-v1`.

## Setup
----
We import useful packages: `gym`, `torch` stuff, etc..

Imports:

In [291]:
import gym

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


from collections import deque  # for memory
from tqdm import tqdm          # for progress bar
import numpy as np


How the game looks (without our agent):

In [292]:
"""env = gym.make('CartPole-v1', render_mode='human')
for _ in tqdm(range(10)):
    state, _ = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        next_state, reward, done, _, _ = env.step(action)
env.close()"""

"env = gym.make('CartPole-v1', render_mode='human')\nfor _ in tqdm(range(10)):\n    state, _ = env.reset()\n    done = False\n    while not done:\n        action = env.action_space.sample()\n        next_state, reward, done, _, _ = env.step(action)\nenv.close()"

## DQN Algorithm
-------------
We train a policy that tries to maximize the discounted,
cumulative reward
$R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_t$, where
$R_{t_0}$ is *return*. The discount, $\gamma$ is the discount, between $0$ and $1$


Q-learning tries to find a function
$Q^*: State \times Action \rightarrow \mathbb{R}$, maximizes rewards:

$\begin{align}\pi^*(s) = \arg\!\max_a \ Q^*(s, a)\end{align}$

However, we don't know $Q^*$. So, we use neural network as a approximators, we can simply create one and train it to resemble $Q^*$.

For our training update rule, we'll use a fact that every $Q$
function for some policy obeys the Bellman equation:

$\begin{align}Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))\end{align}$

The difference between the two sides of the equality is known as the temporal difference error, $\delta$:

$\begin{align}\delta = Q(s, a) - (r + \gamma \max_a Q(s', a))\end{align}$

### Model
---
Make a deep learning based policy model, that takes in a state and outputs an action.
This model will be an attribute of the Agent we make next.

In [293]:
class Model(nn.Module):
     def __init__(self, observation_size, action_size):
         super(Model, self).__init__()
         self.dense1 = nn.Linear(observation_size, 8)
         self.dense2 = nn.Linear(8, 8)
         self.dense3 = nn.Linear(8, 8)
         # had to hardcode it to 4 becuse action_size is 2
         self.dense4 = nn.Linear(8, action_size)


     def forward(self, x):
          x = self.dense1(x)
          x = F.relu(x)
          x = self.dense2(x)
          x = F.relu(x)
          x = self.dense3(x)
          x = F.relu(x)
          x = self.dense4(x)
          return x

     def predict(self,x):
        
          x = torch.tensor(x)
          x = self.forward(x)
          
          return torch.argmax(x, dim=0)   

### DQN Agent
----
We will be using experience replay memory for training our model.
An Agent's memory is as important as its model, and will be another attribute of our agent.
Other appropriate attributes are the hyperparameters (gamma, lr, etc.).
Give the agent a replay method that trains on a batch from its memory.


In [294]:
"""env = gym.make('CartPole-v1')
state, _ = env.reset()"""


"env = gym.make('CartPole-v1')\nstate, _ = env.reset()"

In [295]:
from copy import deepcopy
import random
from turtle import done, st

class Agent:
    def __init__(self, observation_size, action_size):
        self.observation_size=observation_size
        self.action_size = action_size
        self.criterion = nn.MSELoss()
        self.model = Model(observation_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        self.memory = deque([],maxlen=100) # memory that stores N most new transitions
        self.epsilon = 1.0
        # good place to store hyperparameters as attributes

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state,action,reward,next_state,done))
        

    def act(self, state):
        # return an action from the model
        return self.model.predict(state)
        

    def replay(self, batch_size):
        # update model based on replay memory
        # you might want to make a self.train() helper method
        data_set = random.sample(self.memory,batch_size)

        for d in data_set:
            state = torch.tensor(d[0])
            pred = self.model(state)
            target = pred.clone()
            self.train(d,target)

        self.optimizer.zero_grad()
        loss = self.criterion(target, pred)
        loss.backward()
        self.optimizer.step()

    def train(self, data_set, target):
        state = torch.tensor(data_set[0])
        action = torch.tensor(data_set[1])
        reward = torch.tensor(data_set[2])
        next_state = torch.tensor(data_set[3])
        gamma = 0.02
        
        new_state = reward

        if not done:
            best_next_action = torch.max(next_state)
            new_state = reward + gamma * torch.max(self.model(next_state))
            target = new_state
        
        
        
        


### Main Training loop
---
Make a function that takes and environment and an agent, and runs through $n$ episodes.
Remember to call that agent's replay function to learn from its past (once it has a past).


In [296]:
def train(envrioment, agent:Agent, episodes=1000, batch_size=64):  # train for many games
    for _ in tqdm(range(episodes)):
        state, _ = envrioment.reset()
        done = False
        while not done:
            # 1. make a move in game.
            agent.act(state)
            # 2. have the agent remember stuff.
            action = envrioment.action_space.sample()
            next_state, reward, done, _, _ = envrioment.step(action)
            agent.remember(state,action,reward,next_state,done)
            # 3. update state
            state = next_state
            # 4. if we have enough experiences in out memory, learn from a batch with replay.
            if len(agent.memory) >= batch_size:
                agent.replay(batch_size)
    envrioment.close()


### Putting it together
---
We train an agent on the environment:


In [297]:
env = gym.make('CartPole-v1', render_mode='human')
agent = Agent(env.observation_space.shape[0], env.action_space.n)
train(env, agent)
env.close()
torch.save(agent.model.state_dict(), 'model_8x8x8.pth')

100%|██████████| 1000/1000 [08:22<00:00,  1.99it/s]


## Optional (highly recommended): Atari
Adapt your agent to play an Atari game of your choice.
https://www.gymlibrary.dev/environments/atari/air_raid/